In [1]:
# !pip install web3
# !pip install python-dotenv
# !pip install requests

## Imports

In [2]:
from web3 import Web3

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# To load ABI
import json

import requests
import json

## Constants¶

In [3]:
# WETH token contract
WETH_TOKEN = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

# USDT token address
USDT_TOKEN = '0xdAC17F958D2ee523a2206206994597C13D831ec7'

# USDC token contract
USDC_TOKEN = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'

# Path to the ERC20 ABI
ERC20_ABI_PATH = 'abi/erc20.abi.json'

# ERC20 functions
ERC20_FUNCTIONS = ['name', 'symbol', 'decimals']

# Maps Proxy address for the USDC
TOKEN_TO_PROXY = {USDC_TOKEN:'0x43506849D7C04F9138D1A2050bbF3A0c054402dd'}

# Etherscan endpoint
ETHERSCAN_ENDPOINT = 'https://api.etherscan.io/api'

## Load environment variables

In [4]:
dotenv_path = Path('.env/token')
load_dotenv(dotenv_path=dotenv_path)
PROVIDER_URL = os.getenv('PROVIDER_URL')
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')

## Initialize web3 instance and Load ERC20 ABI

In [5]:
# Initialize web3 instance
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))

# Load the previously downloaded ERC20 ABI
with open(ERC20_ABI_PATH) as f:
    erc20_abi = json.load(f)

# Cache token contracts - to avoid repeated calls via Etherscan API; address -> contract
token_to_contract = {}

## Call Token Functions - ERC20

In [6]:
def execute_functions_erc20(address:str) -> list:
    """ Returns a list of results of calling functions in ERC20_FUNCTIONS constant
    Parameters:
    address : str
        address of the token to execute funtions
    Returns:
    list
        A list of a list of function call results
    """
    # Create an instance of the function
    contract = web3.eth.contract(address=address, abi=erc20_abi)

    selected_functions = [x for x in contract.all_functions() if x.name in ERC20_FUNCTIONS]
    # Tuple of function name and the result
    result = [(x.name, x.call()) for x in selected_functions]
    return result

In [7]:
[x for x in execute_functions_erc20(address=WETH_TOKEN)]

[('decimals', 18), ('name', 'Wrapped Ether'), ('symbol', 'WETH')]

## Call Token Functions

In [8]:
def execute_functions(address:str, function_names:list[str]) -> list:
    """ Returns a list of results of calling functions in the given function names
    Parameters:
    address : str
        address of the token to excute funtions
    function_names: list
        list of funcions names we need to execute
    Returns:
    list
        A list of function call results
    """
    # Parameters for etherscan API
    params = {'module':'contract', 'action':'getabi', 'apikey':{ETHERSCAN_API_KEY}}
    # Check for proxy address for the given address
    if address in TOKEN_TO_PROXY:
        # For a proxy to use the proxy address to get the ABI
        params['address'] = TOKEN_TO_PROXY[address]
    else:
        params['address'] = address
    
    # Check the cache first
    if address in token_to_contract:
        contract = token_to_contract[address]
    else:        
        # Call to get the ABI
        response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
        # Create an instance of the function
        contract = web3.eth.contract(address=address, abi=response['result'])
        token_to_contract[address] = contract

    selected_functions = [x for x in contract.all_functions() if x.name in function_names]
    # Tuple of function name and the result
    result = [(x.name, x.call()) for x in selected_functions]
    return result

In [9]:
# [x for x in execute_functions(address=WETH_TOKEN, function_names=ERC20_FUNCTIONS)]
[x for x in execute_functions(address=USDC_TOKEN, function_names=ERC20_FUNCTIONS + ['owner', 'version', 'currency'])]
# [x for x in execute_functions(address=USDT_TOKEN, function_names=ERC20_FUNCTIONS + ['owner'])]

[('currency', 'USD'),
 ('decimals', 6),
 ('name', 'USD Coin'),
 ('owner', '0xFcb19e6a322b27c06842A71e8c725399f049AE3a'),
 ('symbol', 'USDC'),
 ('version', '2')]

## Support method to get Token functions

In [10]:
def get_token_functions(address:str) -> list:
    """ Returns a list of token functions
    Parameters:
    address : str
        address of the token to get info
        
    Returns:
    list
        A lisof all the functions
    """
    # Get ABI of contract; use etherscan API for verifiable smart contracts ABI
    params = {'module':'contract', 'action':'getabi', 'apikey':{ETHERSCAN_API_KEY}}
    # Check for proxy address for the given address
    if address in TOKEN_TO_PROXY:
        # For a proxy to use the proxy address to get the ABI
        params['address'] = TOKEN_TO_PROXY[address]
    else:
        params['address'] = address

    # Check the cache first
    if address in token_to_contract:
        print('found {address} in the cache'.format(address=address))
        contract = token_to_contract[address]
    else:            
        # Call to get the ABI
        response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
        contract = web3.eth.contract(address=address, abi=response['result'])
        token_to_contract[address] = contract
    return contract.all_functions()

In [11]:
get_token_functions(WETH_TOKEN)

[<Function allowance(address,address)>,
 <Function approve(address,uint256)>,
 <Function balanceOf(address)>,
 <Function decimals()>,
 <Function deposit()>,
 <Function name()>,
 <Function symbol()>,
 <Function totalSupply()>,
 <Function transfer(address,uint256)>,
 <Function transferFrom(address,address,uint256)>,
 <Function withdraw(uint256)>]

In [12]:
# get_token_functions(USDT_TOKEN)

In [13]:
# get_token_functions(USDC_TOKEN)